# classes and metaclasses

Tutorials on metaclass
- a lengthy and comprehensive [one](https://www.honeybadger.io/blog/python-instantiation-metaclass/)
- a shorter and simpler [one](https://realpython.com/python-metaclasses/)

A basic [tutorial](https://www.pythonpool.com/python-cls-vs-self/) on static, class, instance, cls and self. 

A tutorial on [super](https://realpython.com/python-super/)

In [ ]:
from debuggable.utils import *
from pprint import pprint
import inspect

## Classes

### how-method: static vs class vs instance methods

In [ ]:
class Sample:
   
    @staticmethod
    def method():
        print('This is a static method')
 
Sample.method()

In [ ]:
class Sample:
    var = "Class Variable"
    
    @classmethod
    def method(cls):
        print(f"this is {cls.var}")
 
Sample.method()

In [ ]:
class Sample:
    def __init__(self, a):
        self.a = a
 
    def method(self):
        print(self.a)
        print("I am an instance method, like `__init__`")
 
obj = Sample(10)
obj.method()

### how-cls and how-self

cls refers to the class, whereas self refers to the instance. Using the cls keyword, we can only access the members of the class, whereas using the self keyword, we can access both the instance variables and the class attributes

In [ ]:
class Person:
    about = 'This class stores the name and age for a person' 
    # class variable/property, no need instance to access it
 
    def __init__(self, name, age): # instance method
        self.name = name # define and set an instance variable inside self
        self.age = age
 
    def details(self): # 
        print(f"Person's name is {self.name} and age is {self.age}")

    def instanceinfo(self):
        print(self.about) # an instance can access class property or variable
        self.info() # an instance can access class method too
        
    @classmethod # class method
    def info(cls): # class method use cls, no need instance to access it
        print(cls.about)
 


## Type and Class

### how-type: `type(obj)` is equivalent to `obj.__class__`

In [ ]:
class Foo: pass

for obj in (1, [2], {"three": 3}, (4), Foo):
    print(type(obj) is obj.__class__)

### how-type: type are the class (or metaclass) of classes like int, float, etc

In [ ]:
for t in int, float, dict, list, tuple, Foo, Foo(), object:
...     print(type(t))

In [ ]:
type(type)

- x is an instance of class Foo.
- Foo is an instance of the type metaclass.
- type is also an instance of the type metaclass, so it is an instance of itself.

![type](https://files.realpython.com/media/class-chain.5cb031a299fe.png)

### how-type: create class dynamically using `type`

You can also call type() with three arguments—`type(<name>, <bases>, <dct>)`:

- `<name>` specifies the class name. This becomes the __name__ attribute of the class.
- `<bases>` specifies a tuple of the base classes from which the class inherits. This becomes the __bases__ attribute of the class.
- `<dct>` specifies a namespace dictionary containing definitions for the class body. This becomes the __dict__ attribute of the class.

#### Example: Create a class without bases and attributes

In [ ]:
Foo = type('Foo', (), {})
class Foo: pass

#### Example: Create a class with bases and attributes

In [ ]:
Bar = type('Bar', (Foo,), dict(attr=100))
class Bar(Foo): attr = 100

#### Example: Create a class without base but attributes and methods

In [ ]:
Foo = type('Foo', (), {'attr': 100, 'attr_val': lambda x : x.attr})
class Foo: 
    attr = 100 
    def attr_val(self): return self.attr

In [ ]:
def f(obj):
    print('attr =', obj.attr)
Foo = type('Foo', (), {'attr': 100,'attr_val': f})

def f(obj):
    print('attr =', obj.attr)
class Foo:
    attr = 100
    attr_val = f

## metaclass: Customize your own type 

### What exactly happening when you call a class?

In [ ]:
class Foo: pass
f = Foo()

In [ ]:
type(Foo) == Foo.__class__ == type # Foo as a class is created by metaclass type

In [ ]:
Foo.__bases__ # Foo inherits from object, but is created by type

To creating a class, interpreter will first run `type.__call__` which will run `obj.__new__` and `obj.__init__`

In [ ]:
def new(cls):
    x = object.__new__(cls) # `__new__` is from object, not type
    x.attr = 100
    return x

Foo.__new__ = new

### Can we customize type by customizing its `__new__`

```python
# Spoiler alert:  This doesn't work!
def new(cls):
    x = type.__new__(cls)
    x.attr = 100
    return x

type.__new__ = new
Traceback (most recent call last):
  File "<pyshell#77>", line 1, in <module>
    type.__new__ = new
TypeError: can't set attributes of built-in/extension type 'type'
```

### how-type: how to create a type or class dynamically

```python
type??
Init signature: type(self, /, *args, **kwargs)
Docstring:     
type(object) -> the object's type
type(name, bases, dict, **kwds) -> a new type
Type:           type
Subclasses:     ABCMeta, EnumMeta, NamedTupleMeta, _TypedDictMeta, _ABC, MetaHasDescriptors, PyCStructType, UnionType, PyCPointerType, PyCArrayType, ...
```

In [ ]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        x.attr = 100
        return x

In [ ]:
type(Meta) == Meta.__class__ == type == Meta.__bases__[0] 
# Meta is created by type and inherited from type

In [ ]:
class Foo(metaclass=Meta):
    pass
pprint(Foo.attr)
pprint(Foo.__bases__) # Foo is inherited from object
pprint(type(Foo) == Foo.__class__ == Meta) # Foo is created by Meta


In [ ]:
F = Meta('Foo', (), {'attr': 100})
pprint(F.attr)
pprint(F.__bases__) # F is inherited from object
pprint(type(F) == F.__class__ == Meta) # F is created by Meta


### Metaclass really necessary?

#### Create a class with class attr using metaclass

In [ ]:
class Meta(type):
    def __init__(
        cls, name, bases, dct
    ):
        cls.attr = 100

In [ ]:
class X(metaclass=Meta):
    pass
X.attr

In [ ]:
Xm = Meta('X', (), {'attr':100})
Xm.attr

In [ ]:
pprint(type(Xm) == Xm.__class__ == Meta)
pprint(Xm.__bases__)

#### Using simple inheritance

In [ ]:
class Base: attr = 100
class X(Base): pass
X.attr

In [ ]:
pprint(type(X) == X.__class__ == type)
pprint(X.__bases__)

#### Using class decorator

In [ ]:
def decorator(cls):
    class NewClass(cls): attr = 100
    return NewClass

@decorator
class X: pass

X.attr

In [ ]:
pprint(type(X) == X.__class__ == type)
pprint(X.__bases__)

## how to make an object callable with `__call__`

In [ ]:
class Human:
    def __init__(self, first_name, last_name):
        print("I am inside __init__ method")
        self.first_name = first_name
        self.last_name = last_name

# The following attempts won't work

#     @staticmethod  
#     def __call__():
#         print("I am inside __call__ method")

#     @classmethod
#     def __call__(cls):
#         print("I am inside __call__ method")

    def __call__(cls):
        print("I am inside __call__ method using cls as args")
        
    def __call__(self):
        print("I am inside __call__ method using self as args")

In [ ]:
h = Human(1,2)
h() # __call__(self) will take priority over __call__(cls), even though both work on its own
try:
    Human()
except TypeError as e: 
    print(e)

In [ ]:
h.__call__()

## all classes inherit from object

In Python3, all classes implicitly inherit from the built-in object base class. 
To specify explicitly `object` as a base class won't be confused with metaclass

In [ ]:
class Human(object): pass
pprint(Human.__class__ == type(Human) == type)
pprint(Human.__bases__)

In [ ]:
class Human(): pass
pprint(Human.__class__ == type(Human) == type)
pprint(Human.__bases__)

## Type as metaclass

In [ ]:
class Human: pass
pprint(type(Human))
pprint(type(type))

![type flow](https://www.honeybadger.io/images/blog/posts/python-instantiation-metaclass/metaclass.png?1660181823)

![Type flow instantiation](https://www.honeybadger.io/images/blog/posts/python-instantiation-metaclass/combined.png?1660181823)

In [ ]:
class Human:
    def __init__(self, a, b):
        self.a = a
        self.b = a

human_obj = Human(1, 2)
assert isinstance(human_obj, Human) 
assert isinstance(human_obj, object) 

## `object.__new__` and overriding it by subclasses

### what `object.__new__` look like

```python
object.__new__??
Signature: object.__new__(*args, **kwargs)
Docstring: Create and return a new object.  See help(type) for accurate signature.
Type:      builtin_function_or_method
```

```python
# cls - is the mandatory argument. Object returned by the __new__ method is of type cls
@staticmethod
def __new__(cls[,...]):
    pass
```

### What overriding `object.__new__` look like

In [ ]:
class Human:
    # no @staticmethod is required when overiding the staticmethod __new__ of object
    def __new__(cls, first_name=None):
        # cls = Human. cls is the class using which the object will be created.
        # Created object will be of type cls.
        # We must call the object class' __new__ to allocate memory
        obj = super().__new__(cls) # This is equivalent to object.__new__(cls)

        # receive obj from object.__new__ and modify it inside Human.__new__
        obj.name = first_name if first_name else "Virat"

        # check the object's class or type
        print(type(obj)) # Prints: <__main__.Human object at 0x103665668>
        # return the object
        return obj

# Create an object
# __init__ method of `object` class will be called.
virat = Human()

print(virat.name)  # Output: Virat

sachin = Human("Sachin")
print(sachin.name)  # Output: Sachin

In [ ]:
class Animal:
    def __new__(cls):
        # cls = Animal, but we don't have to use it.
        # Passing Human class reference instead of Animal class reference
        obj = super().__new__(Human) # This is equivalent to object.__new__(Human)

        print(f"Type of obj: {type(obj)}") # Prints: Type of obj: <class '__main__.Human'>

        # return the object
        return obj

# Create an object
cat = Animal()
# Output:
# Type of obj: <class '__main__.Human'>

type(cat)   # Output: <class '__main__.Human'>

## `object.__init__` and overriding it by subclasses

In [ ]:
class Human:
    # overriding object.__init__ with Human.__init__ below
    def __init__(self, first_name):
        # self = obj. __init__ received obj from Human.__new__ or object.__new__
        self.first_name = first_name
        
        # NEVER return self in __init__
        return self

try:
    human_obj = Human('Virat')
except TypeError as e: 
    print(e)

### Overriding both `__new__` and `__init__`

```python
object.__init__??
Signature:      object.__init__(self, /, *args, **kwargs)
Call signature: object.__init__(*args, **kwargs)
Type:           wrapper_descriptor
String form:    <slot wrapper '__init__' of 'object' objects>
Namespace:      Python builtin
Docstring:      Initialize self.  See help(type(self)) for accurate signature.
```

In [ ]:
class Human:
    def __new__(cls, *args, **kwargs):
        # Here, the __new__ method of the object class must be called to create
        # and allocate the memory to the object
        print("Inside `__new__` method")
        print(f"args arguments {args}")
        print(f"kwargs arguments {kwargs}")

        # The code below calls the __new__ method of the object's class.
        # Object class' __new__ method allocates a memory
        # for the instance and returns that instance
        human_obj = super(Human, cls).__new__(cls)

        print(f"human_obj instance - {human_obj}")
        return human_obj

    # As we have overridden the __init__ method, 
    # the __init__ method of the object class will Not Be Called
    def __init__(self, first_name, last_name):
        print("Inside __init__ method")
        # self = human_obj returned from the __new__ method

        self.first_name = first_name
        self.last_name = last_name

        print(f"human_obj instance inside __init__ {self}: {self.first_name}, {self.last_name}")

human_obj = Human("Virat", "Kohli")

## `type.__call__`

In [ ]:
class type:
    def __call__():
        # Called when class is called i.e. Human()
        print("type's call method")

- Who calls the __new__ and __init__ method?
- Who passes the self object to the __init__ method?
- As the __init__ method is called after the __new__ method, and the __init__ method does not return anything, how does calling the class return the object (i.e., how does calling the Human class return the human_obj object)?

In [ ]:
class Human:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

human_obj = Human("Virat", "Kohli")

As we are talking about CPython, the type class' __call__ method [definition](https://eli.thegreenplace.net/2012/04/16/python-object-creation-sequence) is defined in C language. If we convert it into Python and simplify it, it will look somewhat like this:

In [ ]:
# type's __call__ method which gets called when Human class is called i.e. Human()
def __call__(cls, *args, **kwargs):
    # cls = Human class
    # args = ["Virat", "Kohli"]
    # Calling __new__ method of the Human class, as __new__ method is not defined
    # on Human, __new__ method of the object class is called
    human_obj = cls.__new__(*args, **kwargs)

    # After __new__ method returns the object, __init__ method will only be called if
    # 1. human_obj is not None
    # 2. human_obj is an instance of class Human
    # 3. __init__ method is defined on the Human class
    if human_obj is not None and isinstance(human_obj, cls) and hasattr(human_obj, '__init__'):
        # As __init__ is called on human_obj, self will be equal to human_obj in __init__ method
        human_obj.init(*args, **kwargs)

    return human_obj

![call-new-init](https://www.honeybadger.io/images/blog/posts/python-instantiation-metaclass/object-instantiation-and-creation.png?1660181823)

- If the `__new__` method does not return anything, then __init__ will not be called
- If the `__new__` method did not return human_obj but an integer with value 10, which is not of the Human type; hence, the `__init__` method will not be called. Also, human_obj will not have the reference for the created object, but it will refer to an integer value of 10.


## super and inheritance

### Basic usages of `super`

In [ ]:
class Rectangle:
    # call object.__new__ implicitly
    
    # overriding object.__init__ with the following
    def __init__(self, length, width):
        self.length = length
        self.width = width

    def area(self):
        return self.length * self.width

    def perimeter(self):
        return 2 * self.length + 2 * self.width

# Here we declare that the Square class inherits from the Rectangle class
class Square(Rectangle):
    # call Rectangle.__new__ implicitly

    # Square needs to override Rectangle.__init__ for its uniqueness
    def __init__(self, length):
        # but we can still use Rectangle.__init__ here
        super().__init__(length, length)
        
    # Square can inherit all base class methods and attributes
        
pprint(Square(1).area())
pprint(Square(1).perimeter())

In [ ]:
class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)

class Cube(Square):
    # call implicitly Square.__new__, and Square.__init__
    # Cube inherit all methods and attributes from Square
    
    def surface_area(self):
        face_area = super().area() # use Square.area to build Cube.surface_area
        return face_area * 6

    def volume(self):
        face_area = super().area() # use Square.area to build Cube.surface_area
        return face_area * self.length

### Deep dive in `super`

In [ ]:
class Rectangle:
    def __init__(self, length, width):
        self.length = length
        self.width = width

    def area(self):
        return self.length * self.width

    def perimeter(self):
        return 2 * self.length + 2 * self.width

class Square(Rectangle):
    def __init__(self, length):
        super(Square, self).__init__(length, length) 
        # equivalent to super().__init__(length, length)

In [ ]:
class Cube(Square):
    def surface_area(self):
        face_area = super(Square, self).area()
        return face_area * 6

    def volume(self):
        # jump in inheritance ladder, to search inside Rectangle class not Square class
        face_area = super(Square, self).area()
        return face_area * self.length

#### How to understand `super(Square, self)`

By including an instantiated object like `self` above, `super()` returns a bound method: a method that is bound to the object, which gives the method the object’s context such as any instance attributes. 

If this parameter is not included, the method returned is just a function, unassociated with an object’s context

### `super()` in Multiple Inheritance

Python supports multiple inheritance, in which a subclass can inherit from multiple superclasses that don’t necessarily inherit from each other (also known as sibling classes).
![sibling superclasses](https://files.realpython.com/media/multiple_inheritance.22fc2c1ac608.png)

In [ ]:
class Triangle:
    def __init__(self, base, height):
        self.base = base
        self.height = height

    def area(self):
        return 0.5 * self.base * self.height

In [ ]:
class RightPyramid(Triangle, Square): # note the order of super classes
    def __init__(self, base, slant_height):
        self.base = base
        self.slant_height = slant_height

    def area(self):
        base_area = super().area()
        perimeter = super().perimeter()
        return 0.5 * perimeter * self.slant_height + base_area

In [ ]:
pyramid = RightPyramid(2, 4)
try:
    pyramid.area()
except AttributeError as e:
    print(e)

```python
Traceback (most recent call last):
  File "shapes.py", line 63, in <module>
    print(pyramid.area())
  File "shapes.py", line 47, in area
    base_area = super().area()
  File "shapes.py", line 38, in area
    return 0.5 * self.base * self.height
AttributeError: 'RightPyramid' object has no attribute 'height'
```

### MRO: method resolution order
The method resolution order (or MRO) tells Python how to search for inherited methods.

In [ ]:
RightPyramid.__mro__

In [ ]:
RightPyramid.__base__

Luckily, you have some control over how the MRO is constructed. Just by changing the signature of the RightPyramid class, you can search in the order you want, and the methods will resolve correctly:

In [ ]:
class Triangle:
    def __init__(self, base, height):
        self.base = base
        self.height = height
        super().__init__()

    def tri_area(self):
        return 0.5 * self.base * self.height

class RightPyramid(Square, Triangle):
    def __init__(self, base, slant_height):
        self.base = base
#         self.height = base # add this line to avoid tri_area missing height value
        self.slant_height = slant_height
        super().__init__(self.base)

    def area(self):
        base_area = super().area()
        perimeter = super().perimeter()
        return 0.5 * perimeter * self.slant_height + base_area

    def area_2(self):
        base_area = super().area()
        triangle_area = super().tri_area()
        return triangle_area * 4 + base_area

In [ ]:
pprint(RightPyramid.__base__)

pprint(RightPyramid.__mro__)

In [ ]:
pyramid = RightPyramid(base=2, slant_height=4)
pyramid.area()
try:
    pyramid.area_2()
except AttributeError as e:
    print(e)

```python
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Input In [284], in <cell line: 3>()
      1 pyramid = RightPyramid(base=2, slant_height=4)
      2 pyramid.area()
----> 3 pyramid.area_2()

Input In [282], in RightPyramid.area_2(self)
     22 def area_2(self):
     23     base_area = super().area()
---> 24     triangle_area = super().tri_area()
     25     return triangle_area * 4 + base_area

Input In [282], in Triangle.tri_area(self)
      7 def tri_area(self):
----> 8     return 0.5 * self.base * self.height

AttributeError: 'RightPyramid' object has no attribute 'height'

```

### how-kwargs: Using `**kwargs` to pass values to superclasses

In [ ]:
class Rectangle:
    def __init__(self, length, width, **kwargs):
        self.length = length
        self.width = width
        super().__init__(**kwargs)

    def area(self):
        return self.length * self.width

    def perimeter(self):
        return 2 * self.length + 2 * self.width

# Here we declare that the Square class inherits from 
# the Rectangle class
class Square(Rectangle):
    def __init__(self, length, **kwargs):
        super().__init__(length=length, width=length, **kwargs)

class Cube(Square):
    def surface_area(self):
        face_area = super().area()
        return face_area * 6

    def volume(self):
        face_area = super().area()
        return face_area * self.length

class Triangle:
    def __init__(self, base, height, **kwargs):
        self.base = base
        self.height = height
        super().__init__(**kwargs)

    def tri_area(self):
        return 0.5 * self.base * self.height

class RightPyramid(Square, Triangle):
    def __init__(self, base, slant_height, **kwargs):
        self.base = base
        self.slant_height = slant_height
        kwargs["height"] = slant_height
        kwargs["length"] = base
        super().__init__(base=base, **kwargs)

    def area(self):
        base_area = super().area()
        perimeter = super().perimeter()
        return 0.5 * perimeter * self.slant_height + base_area

    def area_2(self):
        base_area = super().area()
        triangle_area = super().tri_area()
        return triangle_area * 4 + base_area

In [ ]:
pprint(RightPyramid.__base__)
pprint(RightPyramid.__mro__)

In [ ]:
pyramid = RightPyramid(base=2, slant_height=4)
pyramid.area()
pyramid.area_2()

### Mixin

In [ ]:
class Rectangle:
    def __init__(self, length, width):
        self.length = length
        self.width = width

    def area(self):
        return self.length * self.width

class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)

class VolumeMixin: # this is actually a mixin, not perform as a superclass for Cube
    def volume(self):
        return self.area() * self.height

class Cube(VolumeMixin, Square):
    def __init__(self, length):
        super().__init__(length)
        self.height = length

    def face_area(self):
        return super().area()

    def surface_area(self):
        return super().area() * 6

In [ ]:
cube = Cube(2)
cube.surface_area()
cube.volume()

In [ ]:
pprint(Cube.__bases__)
pprint(Cube.__mro__)

#|hide
## Send to Obs

In [ ]:
#|hide
!jupytext --to md /Users/Natsume/Documents/debuggable/fastcore.meta/classes_metaclasses.ipynb
!mv /Users/Natsume/Documents/debuggable/fastcore.meta/classes_metaclasses.md \
/Users/Natsume/Documents/divefastai/Debuggable/jupytext/fastcore.meta/

In [ ]:
!jupyter nbconvert --config /Users/Natsume/Documents/mynbcfg.py --to markdown \
--output-dir /Users/Natsume/Documents/divefastai/Debuggable/nbconvert